# 개요
* GPT활용교육을 들은 내용을 별도로 정리
  * 1일차 : 네이버API 활용 : GPT를 활용한 변수 분석 등
  * 2일차 : beautifulsoup활용한 뉴스 및 증권사리포트 크롤링
  * 3일차 : 뉴스 및 증권사리포트 데이터 전처리 
  * 4일차 : 감성점수 변환
  * 5일차 : 백테스팅 실습

* 특이사항 : 공식문서가 강의내용과 달라져서 **공식문서 기준으로 재정리**

# 3일차 실습내용 정리

## 실습 : OpenAI API 활용

In [ ]:
import json

# OpenAI API key
with open("keys.json", "r") as f:
    data = json.load(f)

open_ai_key = data.get("open_ai_key")

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=open_ai_key)

response = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming"
        }
        ]
        
)

In [ ]:
print(response)

Response(id='', created_at=1748956004.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseOutputMessage(id='msg_', content=[ResponseOutputText(annotations=[], text='Code calls itself back,  \nLayers unfold, truth revealed—  \nEndless loops of thought.', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), service_tier='default', status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=25, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=20, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=45), user=None, store=True)

In [ ]:
print(response.output_text)

Code calls itself back,  
Layers unfold, truth revealed—  
Endless loops of thought.


## 실습 : OPENAI API로 CSV(기사 제목/본문) 확인 + 감성분석용 요약

In [ ]:
# pandas apply에 활용할 OpenAI로 요약해주는 함수 작성

def generate_summary(row):
    prompt = f"""
            아래의 기사 제목과 내용을 바탕으로 간단하고 명확한 요약을 생성해주세요.
            요약은 다음과 같은 요건을 충족해야 합니다:
            1. 기사에 담긴 주요 정보와 메시지를 간결하게 포함할 것.
            2. 긍정적, 부정적 또는 중립적인 뉘앙스를 명확히 드러낼 것. 감정이 모호하지 않도록 작성해주세요.
            3. 독자가 감성 분석을 하기 쉽도록 명료한 문장으로 작성할 것.

            제목: {row['title']}
            내용: {row['summary']}

            아래에 간결한 요약을 작성해주세요:
            """

    client = OpenAI(api_key=open_ai_key)

    response = client.responses.create(
        model="gpt-4o-mini",
        input=[
            {
                "role": "user",
                "content": f"{prompt}"
            }
            ]
            
    )
    return response.output_text

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 50)

In [ ]:
# 기존에 데이터프레임으로 저장해둔 기사 불러오기
import pandas as pd

crawling_news = pd.read_csv(r'2_6_데이터\crawling_news.csv')
crawling_news.head()

,Unnamed: 0,company,title,date,summary
0,0,BGF,"[날개 단 유통 재벌家 아들]④BGF, 10분의1 토막 주가에 한살 터울 형제 승계...",2023-01-31,홍정국 BGF리테일 대표(왼쪽)와 차남 홍정혁 BGF에코머티리얼즈 대표/사진= BG...
1,1,BGF,"""농가 상생에 진심"" CU, 진도 대파 간편식 시리즈 출시",2023-01-31,BGF리테일(282330)이 운영하는 편의점 CU가 지역 농가 상생 프로젝트로 전라...
2,2,BGF,"BGF리테일, 협력사와 경영 특강 및 ESG 경영 선언식 진행",2023-01-30,급변하는 트렌드를 읽고 시장 경쟁력을 강화하는 방법 주제로 협력사 동반성장 특강 진...
3,3,BGF,[브리프] LG생활건강 GS25 세븐일레븐 BGF 그린벳,2023-01-30,"CU, 하이포크 삼겹살 특가 판매 CU 하이포크 삼겹살 할인ㅣBGF CU가 편의점 ..."
4,4,BGF,"[게시판] BGF리테일, 협력사와 경영특강·ESG 선언식",2023-01-29,▲ CU를 운영하는 BGF리테일은 지난 26일 서울 강남구 사옥에서 협력사 대상 경...


In [ ]:
len(crawling_news)

27602

In [ ]:
# API로 기사 요약 진행(API절약을 위해 일부만)

## 10개만 추린 데이터프레임 생성
crawling_news_10rows = crawling_news[:10].copy()
crawling_news_10rows

,Unnamed: 0,company,title,date,summary
0,0,BGF,"[날개 단 유통 재벌家 아들]④BGF, 10분의1 토막 주가에 한살 터울 형제 승계...",2023-01-31,홍정국 BGF리테일 대표(왼쪽)와 차남 홍정혁 BGF에코머티리얼즈 대표/사진= BG...
1,1,BGF,"""농가 상생에 진심"" CU, 진도 대파 간편식 시리즈 출시",2023-01-31,BGF리테일(282330)이 운영하는 편의점 CU가 지역 농가 상생 프로젝트로 전라...
2,2,BGF,"BGF리테일, 협력사와 경영 특강 및 ESG 경영 선언식 진행",2023-01-30,급변하는 트렌드를 읽고 시장 경쟁력을 강화하는 방법 주제로 협력사 동반성장 특강 진...
3,3,BGF,[브리프] LG생활건강 GS25 세븐일레븐 BGF 그린벳,2023-01-30,"CU, 하이포크 삼겹살 특가 판매 CU 하이포크 삼겹살 할인ㅣBGF CU가 편의점 ..."
4,4,BGF,"[게시판] BGF리테일, 협력사와 경영특강·ESG 선언식",2023-01-29,▲ CU를 운영하는 BGF리테일은 지난 26일 서울 강남구 사옥에서 협력사 대상 경...
5,5,BGF,"""신선육 파는 편의점""...CU, 삼겹살 특가 판매",2023-01-30,사진제공=BGF리테일 CU는 근거리 소비 트렌드에 맞춰 신선육 판매를 전국 매장으로...
6,6,BGF,"[실적랭킹] BGF리테일, 지난해 4Q 예상 매출액 증가율 1위 할인∙편의점주",2023-01-31,한국 주식 시장의 할인점∙편의점∙홈쇼핑주 가운데 지난해 4분기 예상 매출액 증가율 ...
7,7,BGF,"전남도, 2022년산 새청무쌀 1만 3000t BGF리테일 공급",2023-01-25,있는 BGF리테일에 2022년산 새청무쌀 1만 3000여t을 공급한다고 25일 밝혔...
8,8,BGF,"[편의점에서는] CU, 진도 대파 간편식 시리즈 출시 外 세븐일레븐, GS25",2023-01-31,(제공: BGF리테일) 31일 편의점 소식을 전한다. ◆“농가 상생에 진심”… CU...
9,9,BGF,"""1400원으로 한 끼 해결""…'짠물 소비족'에 인기 폭발 [박종관의 유통관통]",2023-01-27,27일 BGF리테일이 운영하는 편의점 CU에 따르면 지난해 10~12월 대용량 삼각...


In [ ]:
## 기사 일부만 가져오기 (10행)

crawling_news_10rows['generated_summary'] = crawling_news_10rows.apply(generate_summary, axis=1)
crawling_news_10rows

,Unnamed: 0,company,title,date,summary,generated_summary
0,0,BGF,"[날개 단 유통 재벌家 아들]④BGF, 10분의1 토막 주가에 한살 터울 형제 승계...",2023-01-31,홍정국 BGF리테일 대표(왼쪽)와 차남 홍정혁 BGF에코머티리얼즈 대표/사진= BG...,BGF그룹의 2세 승계가 확정되며 긍정적인 변화가 기대된다. 홍정국 BGF리테일 대...
1,1,BGF,"""농가 상생에 진심"" CU, 진도 대파 간편식 시리즈 출시",2023-01-31,BGF리테일(282330)이 운영하는 편의점 CU가 지역 농가 상생 프로젝트로 전라...,CU가 전라남도 진도군의 특산물인 대파를 활용한 대파 불고기 간편식 시리즈를 출시하...
2,2,BGF,"BGF리테일, 협력사와 경영 특강 및 ESG 경영 선언식 진행",2023-01-30,급변하는 트렌드를 읽고 시장 경쟁력을 강화하는 방법 주제로 협력사 동반성장 특강 진...,BGF리테일은 26일 서울 강남에서 전국 80여 협력사 임직원이 참석한 가운데 경영...
3,3,BGF,[브리프] LG생활건강 GS25 세븐일레븐 BGF 그린벳,2023-01-30,"CU, 하이포크 삼겹살 특가 판매 CU 하이포크 삼겹살 할인ㅣBGF CU가 편의점 ...","LG생활건강, GS25, 세븐일레븐, BGF는 CU의 하이포크 삼겹살 특가 판매를 ..."
4,4,BGF,"[게시판] BGF리테일, 협력사와 경영특강·ESG 선언식",2023-01-29,▲ CU를 운영하는 BGF리테일은 지난 26일 서울 강남구 사옥에서 협력사 대상 경...,BGF리테일이 26일 서울 강남구 사옥에서 협력사를 대상으로 경영 특강 및 ESG ...
5,5,BGF,"""신선육 파는 편의점""...CU, 삼겹살 특가 판매",2023-01-30,사진제공=BGF리테일 CU는 근거리 소비 트렌드에 맞춰 신선육 판매를 전국 매장으로...,"CU가 신선육 판매를 전국 매장으로 확대하며, 삼겹살 특가 할인 행사를 실시한다고 ..."
6,6,BGF,"[실적랭킹] BGF리테일, 지난해 4Q 예상 매출액 증가율 1위 할인∙편의점주",2023-01-31,한국 주식 시장의 할인점∙편의점∙홈쇼핑주 가운데 지난해 4분기 예상 매출액 증가율 ...,BGF리테일이 지난해 4분기 예상 매출액 증가율에서 1위를 기록했다. 기업분석전문 ...
7,7,BGF,"전남도, 2022년산 새청무쌀 1만 3000t BGF리테일 공급",2023-01-25,있는 BGF리테일에 2022년산 새청무쌀 1만 3000여t을 공급한다고 25일 밝혔...,전남도가 2022년산 새청무쌀 1만 3000t을 BGF리테일에 공급하기로 밝혔다. ...
8,8,BGF,"[편의점에서는] CU, 진도 대파 간편식 시리즈 출시 外 세븐일레븐, GS25",2023-01-31,(제공: BGF리테일) 31일 편의점 소식을 전한다. ◆“농가 상생에 진심”… CU...,CU가 진도 대파를 활용한 간편식 시리즈를 출시하며 지역 농가와의 상생을 적극 추진...
9,9,BGF,"""1400원으로 한 끼 해결""…'짠물 소비족'에 인기 폭발 [박종관의 유통관통]",2023-01-27,27일 BGF리테일이 운영하는 편의점 CU에 따르면 지난해 10~12월 대용량 삼각...,"제목: ""1400원으로 한 끼 해결""…'짠물 소비족'에 인기 폭발\n\n요약: BG..."


In [ ]:
# 값 비교용 : 1행 기사 및 본문
print(crawling_news_10rows.iloc[0].title)
print(crawling_news_10rows.iloc[0].summary)

[날개 단 유통 재벌家 아들]④BGF, 10분의1 토막 주가에 한살 터울 형제 승계는 '척척'
홍정국 BGF리테일 대표(왼쪽)와 차남 홍정혁 BGF에코머티리얼즈 대표/사진= BGF 제공 편의점 CU를 운영하는 BGF그룹의 2세 승계가 구체화되고 있다. 홍석조(70) BGF 회장의 장남인 홍정국(41) BGF 대표이사가 그룹 핵심인 편의점 사업을 맡고 차남인 홍정혁(40) BGF에코머티리얼즈 대표가 소재 관련 신사업을...


In [ ]:
# 값 비교용 : 1행 요약 결과
print(crawling_news_10rows.iloc[0].generated_summary)  

BGF그룹의 2세 승계가 확정되며 긍정적인 변화가 기대된다. 홍정국 BGF리테일 대표가 편의점 CU 사업을, 홍정혁 BGF에코머티리얼즈 대표가 신사업을 담당하게 되어 그룹의 미래 계획이 뚜렷해졌다. 현재 주가는 낮지만, 형제 간의 원활한 승계가 잘 진행되고 있다.


In [ ]:
# 저장하여 확인
crawling_news_10rows.to_excel(r'2_6_데이터\crawling_news_summarized.xlsx')

## 처음 제공된 기초 코드 샘플(뉴스 요약)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key ="")

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message.content)

In [ ]:
import pandas as pd
crawling_news = pd.read_csv('뉴스크롤링.csv')
crawling_news.head()

In [ ]:

def generate_summary(row):
    prompt = f"""
            아래의 기사 제목과 내용을 바탕으로 간단하고 명확한 요약을 생성해주세요.
            요약은 다음과 같은 요건을 충족해야 합니다:
            1. 기사에 담긴 주요 정보와 메시지를 간결하게 포함할 것.
            2. 긍정적, 부정적 또는 중립적인 뉘앙스를 명확히 드러낼 것. 감정이 모호하지 않도록 작성해주세요.
            3. 독자가 감성 분석을 하기 쉽도록 명료한 문장으로 작성할 것.

            제목: {row['title']}
            내용: {row['summary']}

            아래에 간결한 요약을 작성해주세요:
            """
    client = OpenAI(api_key ="")
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user","content": prompt}]
    )

    print(completion.choices[0].message.content)

    return completion.choices[0].message.content

In [ ]:
crawling_news['generated_summary'] = crawling_news.apply(generate_summary, axis=1)

## 실습 : 증권사 리포트 pdf의 첫 페이지를 읽어 요약하기

In [ ]:
# 데이터프레임으로 정리된 증권사 리포트 불러오기

import pandas as pd
pd.set_option('display.max_colwidth', 50)

report_df = pd.read_csv(r'2_6_데이터\crawling_report.csv')
report_df.head()

,Unnamed: 0,company,title,securities_company,attachment,date,summarize_text
0,0,LG에너지솔루션,2023년 판매 고객사에 따른 실적 차별화,메리츠증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,2023. 1. 30 \nLG에너지솔루션 373220 \n2023년 판매 고객사...
1,1,삼성SDI,달라지는 업황에 따른 기대감,DS투자증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,"\nBUY (유지) \n \n목표주가 (상향) 890,000원 \n현재주가 (0..."
2,2,삼성SDI,4Q22 Review: 대체적으로 컨센 부합,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Compan y Report \n삼성SDI \n006400 \n ...
3,3,삼성중공업,4Q22 Review: 동트기 전 마지막 성장통,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Company Report \n삼성중공업 \n010140 \n Ja...
4,4,SGC에너지,상한제 규제 리스크 해소 여부에 관심,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \nSGC에너지...


In [ ]:
# 데이터 수 확인 후 실습용으로 일부만 추출
print(len(report_df))

report_df_10rows = report_df[:10].copy()
report_df_10rows

3274


,Unnamed: 0,company,title,securities_company,attachment,date,summarize_text
0,0,LG에너지솔루션,2023년 판매 고객사에 따른 실적 차별화,메리츠증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,2023. 1. 30 \nLG에너지솔루션 373220 \n2023년 판매 고객사...
1,1,삼성SDI,달라지는 업황에 따른 기대감,DS투자증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,"\nBUY (유지) \n \n목표주가 (상향) 890,000원 \n현재주가 (0..."
2,2,삼성SDI,4Q22 Review: 대체적으로 컨센 부합,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Compan y Report \n삼성SDI \n006400 \n ...
3,3,삼성중공업,4Q22 Review: 동트기 전 마지막 성장통,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Company Report \n삼성중공업 \n010140 \n Ja...
4,4,SGC에너지,상한제 규제 리스크 해소 여부에 관심,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \nSGC에너지...
5,5,한전KPS,중장기 실적은 우상향 추세,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n한전KPS (...
6,6,삼성중공업,2023년은 흑자전환의 원년,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n삼성중공업 ...
7,7,삼성SDI,투자 증가세 뚜렷,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n삼성SDI ...
8,8,GS건설,올해 1분기 마진 확인이 필요,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \nGS건설 (...
9,9,삼성SDI,성장에 대한 의구심 불식,한화투자증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,2023년 01월 31일 \n기업분석 \n \n \n \n삼성SDI (006400...


In [ ]:
# (실습용pdf가 미제공되어 다운로드 하는 코드 GPT작성)

import os
import pandas as pd
import requests
from urllib.parse import urlparse

# 저장할 폴더 경로 및 참고할 데이터프레임 설정
save_dir = '2_6_데이터_pdf'
os.makedirs(save_dir, exist_ok=True)

df = report_df_10rows.copy()

# PDF 파일 다운로드
for url in df['attachment']:
    if isinstance(url, str) and url.startswith('https://') and url.lower().endswith('.pdf'):
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()  # HTTP 오류 발생 시 예외 발생
            filename = os.path.basename(urlparse(url).path)
            filepath = os.path.join(save_dir, filename)
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f'다운로드 완료: {filename}')
        except requests.exceptions.RequestException as e:
            print(f'다운로드 실패: {url} - 오류: {e}')
    else:
        print(f'유효하지 않은 PDF URL: {url}')

In [ ]:
# pandas apply에 활용할 OpenAI로 요약해주는 함수 작성

from PyPDF2 import PdfReader
from openai import OpenAI
import json

# OpenAI API key
with open("keys.json", "r") as f:
    data = json.load(f)

open_ai_key = data.get("open_ai_key")

def generate_summary(row):
    # pdf 파일을 가져와서 이를 PyPDF2에 넣어서 text를 추출
    file_name = row['attachment'].split("/")[-1]
    reader = PdfReader(f"2_6_데이터_pdf/{file_name}")
    text = reader.pages[0].extract_text()

    prompt = f"""
    아래의 증권사 리포트 내용을 바탕으로 간단하고 명확한 요약을 생성해주세요.
    요약은 다음과 같은 요건을 충족해야 합니다:
    1. 증권사 리포트에 담긴 주요 정보와 메시지를 간결하게 포함할 것.
    2. 긍정적, 부정적 또는 중립적인 뉘앙스를 명확히 드러낼 것. 감정이 모호하지 않도록 작성해주세요.
    3. 독자가 감성 분석을 하기 쉽도록 명료한 문장으로 작성할 것.

    내용: {text}

    아래에 간결한 요약을 작성해주세요:
    """

    client = OpenAI(api_key=open_ai_key)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user","content": prompt}]
    )

    print(completion.choices[0].message.content)

    return completion.choices[0].message.content

In [ ]:
report_df_10rows['generated_summary'] = report_df_10rows.apply(generate_summary, axis=1)

In [ ]:
report_df_10rows

,Unnamed: 0,company,title,securities_company,attachment,date,summarize_text,generated_summary
0,0,LG에너지솔루션,2023년 판매 고객사에 따른 실적 차별화,메리츠증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,2023. 1. 30 \nLG에너지솔루션 373220 \n2023년 판매 고객사...,"LG에너지솔루션(373220)은 2022년 매출액 25.5조원, 영업이익 1.2조원..."
1,1,삼성SDI,달라지는 업황에 따른 기대감,DS투자증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,"\nBUY (유지) \n \n목표주가 (상향) 890,000원 \n현재주가 (0...","삼성SDI에 대한 투자의견을 ""매수(BUY)""로 유지하며 목표주가는 890,000원..."
2,2,삼성SDI,4Q22 Review: 대체적으로 컨센 부합,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Compan y Report \n삼성SDI \n006400 \n ...,**삼성SDI 리포트 요약** \n삼성SDI(006400)에 대한 투자의견은 'B...
3,3,삼성중공업,4Q22 Review: 동트기 전 마지막 성장통,교보증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n Company Report \n삼성중공업 \n010140 \n Ja...,**삼성중공업 분석 리포트 요약**\n\n- **투자의견**: Buy 유지\n- *...
4,4,SGC에너지,상한제 규제 리스크 해소 여부에 관심,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \nSGC에너지...,2023년 1월 31일 SGC에너지(005090)에 대한 증권사 리포트에서는 목표주...
5,5,한전KPS,중장기 실적은 우상향 추세,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n한전KPS (...,"한전KPS(051600)에 대한 보고서는 긍정적인 투자 전망을 제시하며, 목표주가를..."
6,6,삼성중공업,2023년은 흑자전환의 원년,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n삼성중공업 ...,"삼성중공업에 대한 2023년 투자 리포트에서 목표주가 7,500원을 제시하며 ""매수..."
7,7,삼성SDI,투자 증가세 뚜렷,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \n삼성SDI ...,**삼성SDI (006400) 투자 리포트 요약**\n\n증권사 리포트에서 삼성SD...
8,8,GS건설,올해 1분기 마진 확인이 필요,하나증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,\n \n2023년 1월 31일 I Equity Research \nGS건설 (...,"GS건설(006360)에 대한 2023년 1분기 기업 분석 리포트가 발표되었으며, ..."
9,9,삼성SDI,성장에 대한 의구심 불식,한화투자증권,https://ssl.pstatic.net/imgstock/upload/resear...,2023-01-31,2023년 01월 31일 \n기업분석 \n \n \n \n삼성SDI (006400...,"삼성SDI(006400)에 대한 투자 의견은 'Buy'로 유지되며, 목표주가는 83..."


In [ ]:
# 저장하여 확인
report_df_10rows.to_excel(r'2_6_데이터\crawling_report_summarized.xlsx')

## 처음 제공된 기초 코드 샘플

In [ ]:
from PyPDF2 import PdfReader
from openai import OpenAI
import pandas as pd

report_df = pd.read_csv('리포트크롤링.csv')
report_df.head(5)

def generate_summary(row):
    # pdf 파일을 가져와서 이를 PyPDF2에 넣어서 text를 추출
    file_name = row['attachment'].split("/")[-1]
    reader = PdfReader(f"/리포트/{file_name}")
    text = reader.pages[0].extract_text()

    prompt = f"""
    아래의 증권사 리포트 내용을 바탕으로 간단하고 명확한 요약을 생성해주세요.
    요약은 다음과 같은 요건을 충족해야 합니다:
    1. 증권사 리포트에 담긴 주요 정보와 메시지를 간결하게 포함할 것.
    2. 긍정적, 부정적 또는 중립적인 뉘앙스를 명확히 드러낼 것. 감정이 모호하지 않도록 작성해주세요.
    3. 독자가 감성 분석을 하기 쉽도록 명료한 문장으로 작성할 것.

    내용: {text}

    아래에 간결한 요약을 작성해주세요:
    """

    client = OpenAI(api_key ="")
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user","content": prompt}]
    )

    print(completion.choices[0].message.content)

    return completion.choices[0].message.content

In [ ]:
report_df['generated_summary'] = report_df.apply(generate_summary, axis=1)